In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    LayerNormalization,
    MultiHeadAttention,
    Flatten,
    Add,
)
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-10-25 18:46:05.615843: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-25 18:46:05.617460: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 18:46:05.651203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-25 18:46:05.651665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 18:46:06.384734: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
data = pd.read_csv("../data.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
0,0,2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.396
1,1,2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.385
2,2,2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.379
3,3,2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.368
4,4,2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.375


In [3]:
def preprocess(data):
    data.drop("Unnamed: 0.1", axis=1, inplace=True)
    data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
    data["Date"] = pd.to_datetime(data["Date"])
    data.set_index("Date", inplace=True)
    data.replace(0, np.nan, inplace=True)
    display(data)
    print("Filling missing Values: ")
    display(data.interpolate(method="linear", limit_direction="forward"))
    data.interpolate(method="linear", limit_direction="forward", inplace=True)
    return data

In [4]:
data = preprocess(data)

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.9133,1.4419,1.7200,1.0377,6.8273,5.1597,7.7555,46.27,92.55,3.3960
2010-01-05,0.9143,1.4402,1.7296,1.0371,6.8258,5.1668,7.7564,46.13,91.48,3.3850
2010-01-06,0.9189,1.4404,1.7292,1.0333,6.8272,5.1638,7.7546,45.72,92.53,3.3790
2010-01-07,0.9168,1.4314,1.7409,1.0351,6.8280,5.1981,7.7539,45.67,93.31,3.3680
2010-01-08,0.9218,1.4357,1.7342,1.0345,6.8274,5.1827,7.7553,45.50,92.70,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.6978,1.1174,4.0507,1.3073,6.9954,6.6829,7.7874,71.45,109.47,4.1260
2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filling missing Values: 


,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Date,,,,,,,,,,
2010-01-04,0.913300,1.441900,1.720000,1.0377,6.8273,5.1597,7.755500,46.27,92.550000,3.3960
2010-01-05,0.914300,1.440200,1.729600,1.0371,6.8258,5.1668,7.756400,46.13,91.480000,3.3850
2010-01-06,0.918900,1.440400,1.729200,1.0333,6.8272,5.1638,7.754600,45.72,92.530000,3.3790
2010-01-07,0.916800,1.431400,1.740900,1.0351,6.8280,5.1981,7.753900,45.67,93.310000,3.3680
2010-01-08,0.921800,1.435700,1.734200,1.0345,6.8274,5.1827,7.755300,45.50,92.700000,3.3750
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.697800,1.117400,4.050700,1.3073,6.9954,6.6829,7.787400,71.45,109.470000,4.1260
2019-12-28,0.698667,1.118833,4.038867,1.3068,6.9924,6.6749,7.786833,71.40,109.263333,4.1191
2019-12-29,0.699533,1.120267,4.027033,1.3063,6.9894,6.6669,7.786267,71.35,109.056667,4.1122


In [5]:
LOOK_BACK = 30
PREDICT_DAY = 1
SPLIT_RATIO = 0.8

In [6]:
def Create_Data(
    data, lookback=LOOK_BACK, pred_len=PREDICT_DAY, split_ratio=SPLIT_RATIO
):
    if lookback < 2:
        print("ERROR: Lookback too small")
        return -1

    # declarations

    x = {}
    y = {}
    xtr = {}
    xt = {}
    ytr = {}
    yt = {}
    scalers = {}

    # Creating stepped data

    for i in data.columns:
        xtemp = pd.DataFrame(data[i])
        for j in range(1, lookback + 1):
            xtemp[i + str(j)] = data[i].shift(-1 * j)
        x[i] = xtemp.dropna()

    # Splitting data into x and y

    for i in x.keys():
        y[i] = pd.DataFrame(x[i].iloc[:, -pred_len])
        x[i] = x[i].iloc[:, :-pred_len]

    # Normalizing x and y values

    for i in x.keys():
        scalers[i + "_x"] = MinMaxScaler(feature_range=(0, 1))
        x[i] = scalers[i + "_x"].fit_transform(x[i])
        scalers[i + "_y"] = MinMaxScaler(feature_range=(0, 1))
        y[i] = scalers[i + "_y"].fit_transform(y[i])

    # setting train and test sizes

    tr_len = int(split_ratio * y["India"].shape[0])
    t_len = y["India"].shape[0] - tr_len

    # creating training and testing data

    for i in x.keys():
        xtr[i] = x[i][:tr_len]
        ytr[i] = y[i][:tr_len]
        xt[i] = x[i][-t_len:]
        yt[i] = y[i][-t_len:]

    # returning pertinent data

    return x, y, xtr, xt, ytr, yt, scalers

In [7]:
x, y, xtr, xt, ytr, yt, scalers = Create_Data(data)

In [8]:
def transformer_encoder(sequence_length, feature_dim, num_transformer_blocks=3):
    inputs = Input(shape=(sequence_length, feature_dim))
    x = inputs

    for _ in range(num_transformer_blocks):
        # Transformer Block with multi-head attention
        query = LayerNormalization(epsilon=1e-6)(x)
        key = LayerNormalization(epsilon=1e-6)(x)
        value = LayerNormalization(epsilon=1e-6)(x)

        attn_output = MultiHeadAttention(num_heads=4, key_dim=feature_dim, dropout=0.3)(
            query, value, key=key
        )
        x = Add()([x, attn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

        # Feed Forward Network inside transformer
        ffn_output = Dense(128, activation="relu")(x)
        ffn_output = Dropout(0.3)(ffn_output)
        ffn_output = Dense(64, activation="relu")(ffn_output)
        ffn_output = Dense(feature_dim)(ffn_output)
        x = Add()([x, ffn_output])  # Residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

    x = Flatten()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation="relu")(x)
    outputs = Dense(PREDICT_DAY)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model


def Create_model(x, y, lookback=LOOK_BACK, Pred_size=PREDICT_DAY):
    models = {}
    for i in x.keys():
        reshaped_x = np.reshape(x[i], (x[i].shape[0], 1, x[i].shape[1]))
        models[i] = transformer_encoder(1, reshaped_x.shape[2])
        print(i)
    return models

In [9]:
m = Create_model(x, y)

Australia
Europe
Brazil
Canada
China
Denmark
Hong Kong
India
Japan
Malaysia


In [10]:
def Execute_model(model, xtr, ytr, xt, yt, scaler):
    MAPE = {}
    MAE = {}
    MSE = {}
    MSE_Actual = {}
    MAE_Actual = {}
    MAPE_Actual = {}
    for i in model.keys():
        print(i)
        reshaped_xtr = np.reshape(xtr[i], (xtr[i].shape[0], 1, xtr[i].shape[1]))
        reshaped_xt = np.reshape(xt[i], (xt[i].shape[0], 1, xt[i].shape[1]))

        es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
        reduce_lr = ReduceLROnPlateau(
            monitor="val_loss", factor=0.2, patience=5, min_lr=0.0001
        )

        model[i].fit(
            reshaped_xtr,
            ytr[i],
            epochs=100,
            batch_size=64,
            verbose=1,
            validation_split=0.2,
            callbacks=[es, reduce_lr],
        )

        temp = model[i].predict(reshaped_xt)
        pred = scaler[i + "_y"].inverse_transform(temp)
        act = scaler[i + "_y"].inverse_transform(yt[i])

        MSE[i] = mean_squared_error(yt[i], temp)
        MAE[i] = mean_absolute_error(yt[i], temp)
        MAPE[i] = mean_absolute_percentage_error(yt[i], temp)
        
        MSE_Actual[i] = mean_squared_error(act, pred)
        MAE_Actual[i] = mean_absolute_error(act, pred)
        MAPE_Actual[i] = mean_absolute_percentage_error(act, pred)
        
    # Tabulating Data
    
    metric = pd.DataFrame([MSE, MAE, MAPE])
    metric["Metric"] = ["MSE", "MAE", "MAPE"]
    metric.set_index("Metric", inplace=True)
    
    results = pd.DataFrame([MSE_Actual, MAE_Actual, MAPE_Actual])
    results["Metric"] = ["MSE", "MAE", "MAPE"]
    results.set_index("Metric", inplace=True)
    
    return metric, results

In [11]:
metric, result = Execute_model(m, xtr, ytr, xt, yt, scalers)

Australia
Epoch 1/100
37/37 [==============================] - 6s 21ms/step - loss: 0.1509 - val_loss: 0.0104 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 0s 6ms/step - loss: 0.0976 - val_loss: 0.0198 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 0s 6ms/step - loss: 0.0840 - val_loss: 0.0242 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 0s 6ms/step - loss: 0.0832 - val_loss: 0.0201 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 0s 7ms/step - loss: 0.0783 - val_loss: 0.0195 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 0s 7ms/step - loss: 0.0770 - val_loss: 0.0128 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 0s 7ms/step - loss: 0.0786 - val_loss: 0.0235 - lr: 2.0000e-04
Epoch 8/100
37/37 [==============================] - 0s 7ms/step - loss: 0.0752 - val_loss: 0.0207 - lr: 2.0000e-04
Epoch 9/100
37/37 [==============================] - 0s 7ms/step - loss: 0.0762 - val

37/37 [==============================] - 0s 10ms/step - loss: 0.0497 - val_loss: 0.2465 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 0s 10ms/step - loss: 0.0440 - val_loss: 0.2205 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 0s 10ms/step - loss: 0.0428 - val_loss: 0.2075 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 0s 10ms/step - loss: 0.0406 - val_loss: 0.1905 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0394 - val_loss: 0.1794 - lr: 0.0010
Epoch 8/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0389 - val_loss: 0.1715 - lr: 0.0010
Epoch 9/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0389 - val_loss: 0.1594 - lr: 0.0010
Epoch 10/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0376 - val_loss: 0.1654 - lr: 0.0010
Epoch 11/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0370 - val_loss: 0.1511 - lr: 

37/37 [==============================] - 0s 11ms/step - loss: 0.0556 - val_loss: 0.1751 - lr: 2.0000e-04
Epoch 41: early stopping
23/23 [==============================] - 1s 3ms/step
Hong Kong
Epoch 1/100
37/37 [==============================] - 7s 28ms/step - loss: 0.1030 - val_loss: 0.1093 - lr: 0.0010
Epoch 2/100
37/37 [==============================] - 0s 10ms/step - loss: 0.0311 - val_loss: 0.1150 - lr: 0.0010
Epoch 3/100
37/37 [==============================] - 0s 10ms/step - loss: 0.0255 - val_loss: 0.1196 - lr: 0.0010
Epoch 4/100
37/37 [==============================] - 0s 12ms/step - loss: 0.0232 - val_loss: 0.1236 - lr: 0.0010
Epoch 5/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0215 - val_loss: 0.1139 - lr: 0.0010
Epoch 6/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0209 - val_loss: 0.1146 - lr: 0.0010
Epoch 7/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0200 - val_loss: 0.1137 - lr: 2.0000e-04
Epoch 8/100


37/37 [==============================] - 0s 11ms/step - loss: 0.0967 - val_loss: 0.1181 - lr: 0.0010
Epoch 24/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0978 - val_loss: 0.0977 - lr: 0.0010
Epoch 25/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0978 - val_loss: 0.0969 - lr: 0.0010
Epoch 26/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0965 - val_loss: 0.1001 - lr: 0.0010
Epoch 27/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0970 - val_loss: 0.0931 - lr: 0.0010
Epoch 28/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0975 - val_loss: 0.0920 - lr: 0.0010
Epoch 29/100
37/37 [==============================] - 0s 12ms/step - loss: 0.0973 - val_loss: 0.1021 - lr: 0.0010
Epoch 30/100
37/37 [==============================] - 0s 12ms/step - loss: 0.0963 - val_loss: 0.0952 - lr: 0.0010
Epoch 31/100
37/37 [==============================] - 0s 11ms/step - loss: 0.0975 - val_loss: 0.0987 

In [12]:
result

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,0.011162,0.005668,3.053410,0.054169,0.225636,0.536855,0.006486,212.827525,207.384275,0.672281
MAE,0.099817,0.067832,1.726347,0.230234,0.413969,0.696002,0.079232,14.359100,14.249336,0.812620
MAPE,0.140835,0.059992,0.451010,0.175268,0.059964,0.105982,0.010108,0.205663,0.129639,0.198361


In [13]:
metric

,Australia,Europe,Brazil,Canada,China,Denmark,Hong Kong,India,Japan,Malaysia
Metric,,,,,,,,,,
MSE,5.959041e-02,0.027988,0.412136,0.204716,0.174108,0.115913,0.640932,0.229839,0.083420,0.276604
MAE,2.306300e-01,0.150738,0.634243,0.447577,0.363641,0.323406,0.787598,0.471873,0.285787,0.521244
MAPE,1.981143e+12,0.776082,0.758248,0.626467,0.504829,0.453789,0.908359,0.557128,0.417781,0.704402


In [14]:
import json
import os

# Define the filename
filename = "./results.json"

# Check if the file exists
if os.path.isfile(filename):
    # If the file exists, load the existing data
    with open(filename, "r") as f:
        data = json.load(f)
else:
    # If the file doesn't exist, create an empty dictionary
    data = {}

# Add the result to the dictionary
data["Transformer"] = metric.to_dict()

# Write the dictionary to the file
with open(filename, "w") as f:
    json.dump(data, f)